<a href="https://colab.research.google.com/github/kpaulfran/sapceship_titanic/blob/main/sapceship_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/kpaulfran/sapceship_titanic/main/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/kpaulfran/sapceship_titanic/main/test.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
train.shape

(8693, 14)

1) Fuente de datos

Kaggle: https://www.kaggle.com/competitions/spaceship-titanic/data?select=train.csv

2) Breve descripción de datos

set of personal records recovered from the ship's damaged computer system.

3) ¿Cuál es el objetivo?

Predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. 

4) ¿Este es un problema de clasificación o regresión?

classifier

5) ¿Cuántas características hay?

14

6) ¿Cuántas filas de datos hay?

8693

7) ¿Qué retos, si es que hay alguno, prevén en la limpieza, exploración o modelización con este conjunto de datos?

Habra que realizar imputaciones en ciertas variables ya que superan el 1% de datos missing